## Analytics Use Cases

The following sections outlines the multiple personas in the organization, each will have his own set of needs.

### Sales

As sales manager I want to better understand how my products are used by my customers, main day/hour of use, understand the trends, and ensuring my current product match what my customer is using.
for example if I have a product of 50K units that is my most sold product and the overage on this product is beyond a threshold I defined - I need to know.

- revenu by product

- usage by product

- overage per product

other aggregations based on meta data of events?

### Operational

As an operational manager, I want to confirm total numbers of events my customer are generating, ensuring the meetering capacity can sustain the current load, and plan for future load.

- events totals (subscriptions, all events summerized).

- forcasting of totals, per day/hour.

- aggregation per subscription types.

### Customer Success

As part of the Customer Success jurney, an Account executive might want to share reports with customers, or become aware of the usage when speaking to his customers.

- aggregation per subscription - consumption trends (dates/hours) how am i consuming the services?

- Churn risk per subscription - ability to recognize customers who's behaviour might be an indication that they are either checking alternatives or going to stop using the product.

- Suggest potential other/addtional products to reduce cost.

## Setup

### Mounting your storage

This is to be run once per workspace. 
The scope created with name: _'meter-scope'_
Secrets created manually in akv:

- meter-sa-clientid
- meter-sa-secret

#### Reference
https://docs.microsoft.com/en-us/azure/databricks/security/secrets/

https://docs.microsoft.com/en-us/azure/databricks/data/data-sources/azure/azure-storage#mount-azure-blob

Setting the authrization end point, which is tied to your tenant id

In [ ]:
tenant = '<Your tenant id>'
tenant_end_point = f'https://login.microsoftonline.com/{tenant}/oauth2/token'

### Create required configuration
Using the secret scope and secrets from AKV<br>
When creating the scope, databricks also creates an access policy of _Get,List_ to the AKV.

In [ ]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": dbutils.secrets.get(scope = "meter-scope", key = "meter-sa-clientid"),
          "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope = "meter-scope", key = "meter-sa-secret"),
          "fs.azure.account.oauth2.client.endpoint": tenant_end_point}

### Mount to your storage
Note, the service principal needs to have proper authorization on the storage account.

In [ ]:
container_name = '<your container>'
mount_name = '<your mount name>'
storage_name = '<your storage account name (data lake gen2)>'

In [ ]:

dbutils.fs.mount(
  source = f'abfss://{container_name}@{storage_name}.dfs.core.windows.net/',
  mount_point = f'/mnt/{mount_name}',
  extra_configs = configs)

### Optional - unmount
In case you no longer require the mount, you can unmount it
```
dbutils.fs.unmount(f'/mnt/{mount_name}')
```

#### List files

The ```snapshot``` is a directory created by the metering application

In [ ]:
dbutils.fs.ls(f'/mnt/{mount_name}/snapshots')

#### Reading Capture Content using SQL

Following are couple of examples on how we can query the content of the files directly. In this scenario, we don't even need to provide schema, knowing the structure of the ```Body``` field we can access the values directly, and even cast it to specific type.
> Note, that you can cast to specific type in one command.

In [ ]:

%sql
-- This query shows whats the string content of the avro payload (there is an explicit casting from base64 to string)
-- Note: the path has to be absolute, therefore the name of the mount is part of it, change it to the name you selected.
select string(Body)
from AVRO.`/mnt/martin_raw/capture`



In [ ]:
%sql
--  This query scans the entire directory, it counts number of messages by type
-- Note: the path has to be absolute, therefore the name of the mount is part of it, change it to the name you selected.
select string(Body):type, count(*)
from AVRO.`/mnt/martin_raw/capture`
group by 1

In [ ]:
%sql
--  This query scans the entire directory, it filter out based on the partitioning scheme and also on specific value from the ```Body``` field.
--  it is also filtering any 'null' messages
-- Note: the path has to be absolute, therefore the name of the mount is part of it, change it to the name you selected.

select *
from AVRO.`/mnt/martin_raw/capture`
where string(Body):type is not null and string(Body):value.meterName is not null and p=1

### Operational Queries

#### Counting events - sample criteria

In [ ]:
%sql
-- This query counts all rows from partition '1' and ensuring the meterName is not empty
-- Note: the path has to be absolute, therefore the name of the mount is part of it, change it to the name you selected.

select count(*) 
from AVRO.`/mnt/martin_raw/capture`
where p == 1 and string(Body):value.meterName is not null 

In [ ]:
%sql
-- This query aggregate all meter name and the quantity per meter name
-- Note: the path has to be absolute, therefore the name of the mount is part of it, change it to the name you selected.

select string(Body):value.meterName , sum(string(body):value.quantity::DOUBLE) as sum
from AVRO.`/mnt/martin_raw/capture/`
where string(Body):type is not null and p == 1 and string(Body):value.meterName is not null 
group by 1

#### Date based aggregation

In [ ]:
%sql
-- This query aggregate all meter name and the quantity per meter name occuring on: May 2022
-- Note: the path has to be absolute, therefore the name of the mount is part of it, change it to the name you selected.

select string(Body):value.meterName , sum(string(body):value.quantity::DOUBLE) as sum
from AVRO.`/mnt/martin_raw/capture/`
where string(Body):type is not null and y == 2022 and m==05 
group by 1

#### Snapshot queries

The structure of the snap shot is defined [here](https://github.com/microsoft/metered-billing-accelerator#state-file-for-a-single-partition).

In [ ]:
%sql
-- This query is pulling all the latest JSON content from all partitions
-- Note: the path has to be absolute, therefore the name of the mount is part of it, change it to the name you selected.
          
select meters
from JSON.`/mnt/martin_raw/snapshots/**/latest.json`

